# Feature Extraction Benchmarks
---

This walkthrough serves as a benchmark for comparing `functime` with `tsfresh` feature extraction functions. We begin the analysis by evaluating the speed of feature extraction across time series of three different sizes: 100K, 1M, and 9M. Next, we assess the speed in a groupby and aggregation context, making a performance comparison between functime with polats and tsfresh using pandas.

In [18]:
%%capture
%pip install perfplot
%pip install pandas
%pip install tsfresh
%pip install functime

In [31]:
from typing import Callable

import pandas as pd
import perfplot
import polars as pl
from tsfresh.feature_extraction import feature_calculators as tsfresh

from functime import feature_extractors as fe

In [59]:
pl.Config.set_tbl_rows(100)
pl.Config.set_fmt_str_lengths(60)
pl.Config.set_tbl_hide_column_data_types(True)

polars.config.Config

## 1. Setup for the comparison
---
We are using the M4 dataset. We create a `pd.DataFrame` and `pl.DataFrame` and we define a list of dictionnary with the following structure:
<br>
(<br>
&emsp;  `<functime_function>`,<br>
&emsp;  `<tsfresh_function>`,<br>
&emsp;  `<functime_parameters>`,<br>
&emsp;   `<tsfresh_parameters>`<br>
)<br>

In [32]:
_M4_DATASET = "../../data/m4_1d_train.parquet"

DF_PANDAS = (
    pd.melt(pd.read_parquet(_M4_DATASET))
    .drop(columns=["variable"])
    .dropna()
    .reset_index(drop=True)
)
DF_PL_EAGER = (
    pl.read_parquet(_M4_DATASET).drop("V1").melt().drop("variable").drop_nulls()
)
DF_PL_LAZY = DF_PL_EAGER.lazy()

In [33]:
FUNC_PARAMS_BENCH = [
    (fe.absolute_energy, tsfresh.abs_energy, {}, {}),
    (fe.absolute_maximum, tsfresh.absolute_maximum, {}, {}),
    (fe.absolute_sum_of_changes, tsfresh.absolute_sum_of_changes, {}, {}),
    (
        fe.approximate_entropy,
        tsfresh.approximate_entropy,
        {"run_length": 2, "filtering_level": 0.5},
        {"m": 2, "r": 0.5},
    ),
    # (fe.augmented_dickey_fuller, tsfresh.augmented_dickey_fuller, "param")
    (fe.autocorrelation, tsfresh.autocorrelation, {"n_lags": 4}, {"lag": 4}),
    (
        fe.autoregressive_coefficients,
        tsfresh.ar_coefficient,
        {"n_lags": 4},
        {"param": [{"coeff": i, "k": 4}] for i in range(5)},
    ),
    (fe.benford_correlation2, tsfresh.benford_correlation, {}, {}),
    (fe.benford_correlation, tsfresh.benford_correlation, {}, {}),
    (fe.binned_entropy, tsfresh.binned_entropy, {"bin_count": 10}, {"max_bins": 10}),
    (fe.c3, tsfresh.c3, {"n_lags": 10}, {"lag": 10}),
    (
        fe.change_quantiles,
        tsfresh.change_quantiles,
        {"q_low": 0.1, "q_high": 0.9, "is_abs": True},
        {"ql": 0.1, "qh": 0.9, "isabs": True, "f_agg": "mean"},
    ),
    (fe.cid_ce, tsfresh.cid_ce, {"normalize": True}, {"normalize": True}),
    (fe.count_above, tsfresh.count_above, {"threshold": 0.0}, {"t": 0.0}),
    (fe.count_above_mean, tsfresh.count_above_mean, {}, {}),
    (fe.count_below, tsfresh.count_below, {"threshold": 0.0}, {"t": 0.0}),
    (fe.count_below_mean, tsfresh.count_below_mean, {}, {}),
    # (fe.cwt_coefficients, tsfresh.cwt_coefficients, {"widths": (1, 2, 3), "n_coefficients": 2},{"param": {"widths": (1, 2, 3), "coeff": 2, "w": 1}}),
    (
        fe.energy_ratios,
        tsfresh.energy_ratio_by_chunks,
        {"n_chunks": 6},
        {"param": [{"num_segments": 6, "segment_focus": i} for i in range(6)]},
    ),
    (fe.first_location_of_maximum, tsfresh.first_location_of_maximum, {}, {}),
    (fe.first_location_of_minimum, tsfresh.first_location_of_minimum, {}, {}),
    # (fe.fourier_entropy, tsfresh.fourier_entropy, {"n_bins": 10}, {"bins": 10}),
    # (fe.friedrich_coefficients, tsfresh.friedrich_coefficients, {"polynomial_order": 3, "n_quantiles": 30}, {"params": [{"m": 3, "r": 30}]}),
    (fe.has_duplicate, tsfresh.has_duplicate, {}, {}),
    (fe.has_duplicate_max, tsfresh.has_duplicate_max, {}, {}),
    (fe.has_duplicate_min, tsfresh.has_duplicate_min, {}, {}),
    (
        fe.index_mass_quantile,
        tsfresh.index_mass_quantile,
        {"q": 0.5},
        {"param": [{"q": 0.5}]},
    ),
    (
        fe.large_standard_deviation,
        tsfresh.large_standard_deviation,
        {"ratio": 0.25},
        {"r": 0.25},
    ),
    (fe.last_location_of_maximum, tsfresh.last_location_of_maximum, {}, {}),
    (fe.last_location_of_minimum, tsfresh.last_location_of_minimum, {}, {}),
    # (fe.lempel_ziv_complexity, tsfresh.lempel_ziv_complexity, {"n_bins": 5}, {"bins": 5}),
    (
        fe.linear_trend,
        tsfresh.linear_trend,
        {},
        {
            "param": [
                {"attr": "pvalue"},
                {"attr": "rvalue"},
                {"attr": "intercept"},
                {"attr": "slope"},
                {"attr": "stderr"},
            ]
        },
    ),
    (fe.longest_streak_above_mean, tsfresh.longest_strike_above_mean, {}, {}),
    (fe.longest_streak_below_mean, tsfresh.longest_strike_below_mean, {}, {}),
    (fe.mean_abs_change, tsfresh.mean_abs_change, {}, {}),
    (fe.mean_change, tsfresh.mean_change, {}, {}),
    (
        fe.mean_n_absolute_max,
        tsfresh.mean_n_absolute_max,
        {"n_maxima": 20},
        {"number_of_maxima": 20},
    ),
    (
        fe.mean_second_derivative_central,
        tsfresh.mean_second_derivative_central,
        {},
        {},
    ),
    (
        fe.number_crossings,
        tsfresh.number_crossing_m,
        {"crossing_value": 0.0},
        {"m": 0.0},
    ),
    (fe.number_cwt_peaks, tsfresh.number_cwt_peaks, {"max_width": 5}, {"n": 5}),
    (fe.number_peaks, tsfresh.number_peaks, {"support": 5}, {"n": 5}),
    # (fe.partial_autocorrelation, tsfresh.partial_autocorrelation, "param"),
    (
        fe.percent_reoccurring_values,
        tsfresh.percentage_of_reoccurring_values_to_all_values,
        {},
        {},
    ),
    (
        fe.percent_reoccurring_points,
        tsfresh.percentage_of_reoccurring_datapoints_to_all_datapoints,
        {},
        {},
    ),
    (
        fe.permutation_entropy,
        tsfresh.permutation_entropy,
        {"tau": 1, "n_dims": 3},
        {"tau": 1, "dimension": 3},
    ),
    (
        fe.range_count,
        tsfresh.range_count,
        {"lower": 0, "upper": 9, "closed": "none"},
        {"min": 0, "max": 9},
    ),
    (fe.ratio_beyond_r_sigma, tsfresh.ratio_beyond_r_sigma, {"ratio": 2}, {"r": 2}),
    (
        fe.ratio_n_unique_to_length,
        tsfresh.ratio_value_number_to_time_series_length,
        {},
        {},
    ),
    (fe.root_mean_square, tsfresh.root_mean_square, {}, {}),
    (fe.sample_entropy, tsfresh.sample_entropy, {}, {}),
    (
        fe.spkt_welch_density,
        tsfresh.spkt_welch_density,
        {"n_coeffs": 10},
        {"param": [{"coeff": i} for i in range(10)]},
    ),
    (fe.sum_reoccurring_points, tsfresh.sum_of_reoccurring_data_points, {}, {}),
    (fe.sum_reoccurring_values, tsfresh.sum_of_reoccurring_values, {}, {}),
    (
        fe.symmetry_looking,
        tsfresh.symmetry_looking,
        {"ratio": 0.25},
        {"param": [{"r": 0.25}]},
    ),
    (
        fe.time_reversal_asymmetry_statistic,
        tsfresh.time_reversal_asymmetry_statistic,
        {"n_lags": 3},
        {"lag": 3},
    ),
    (fe.variation_coefficient, tsfresh.variation_coefficient, {}, {}),
    (fe.var_gt_std, tsfresh.variance_larger_than_standard_deviation, {}, {}),
]

## 2 Benchmark core functions
---
Benchmark core function for time series' length of 100_000, 1_000_000 and 9_000_000. (Except 10_000 for `approximate_entropy` and 10_000/100_000 for `number_cwt_peaks` and `sample_entropy`). `all_benchmarks()` iterates through the elements in the `FUNC_PARAMS_BENCH` list and invoke `benchmark()` for each function.

In [34]:
def benchmark(
    f_feat: Callable, ts_feat: Callable, f_params: dict, ts_params: dict, is_expr: bool
):
    if f_feat.__name__ == "approximate_entropy":
        n_range = [10_000]
    elif f_feat.__name__ in ("number_cwt_peaks", "sample_entropy"):
        n_range = [10_000, 100_000]
    else:
        n_range = [10_000, 100_000, 1_000_000, 9_000_000]
    benchmark = perfplot.bench(
        setup=lambda n: (DF_PL_EAGER.head(n), DF_PANDAS.head(n)),
        kernels=[
            lambda x, _y: f_feat(x["value"], **f_params)
            if not is_expr
            else x.select(f_feat(pl.col("value"), **f_params)),
            lambda _x, y: ts_feat(y["value"], **ts_params),
        ],
        n_range=n_range,
        equality_check=False,
        labels=["functime", "tsfresh"],
    )
    return benchmark

In [35]:
def all_benchmarks(params: list[tuple], is_expr: bool) -> list:
    bench_df = pl.DataFrame(
        schema={
            "Feature name": pl.Utf8,
            "n": pl.Int64,
            "functime (ms)": pl.Float64,
            "tfresh (ms)": pl.Float64,
            "diff (ms)": pl.Float64,
            "diff %": pl.Float64,
            "speedup": pl.Float64,
        }
    )
    for x in params:
        try:
            f_feat = x[0]
            print(f"Feature: {f_feat.__name__}")
            bench = benchmark(
                f_feat=f_feat,
                ts_feat=x[1],
                f_params=x[2],
                ts_params=x[3],
                is_expr=is_expr,
            )
            bench_df = pl.concat(
                [
                    pl.DataFrame(
                        {
                            "Feature name": [x[0].__name__] * len(bench.n_range),
                            "n": bench.n_range,
                            "functime (ms)": bench.timings_s[0] * 1_000,
                            "tfresh (ms)": bench.timings_s[1] * 1_000,
                            "diff (ms)": (bench.timings_s[0] - bench.timings_s[1])
                            * 1_000,
                            "diff %": 100
                            * (bench.timings_s[0] - bench.timings_s[1])
                            / bench.timings_s[1],
                            "speedup": bench.timings_s[1] / bench.timings_s[0],
                        }
                    ),
                    bench_df,
                ]
            )
        except ValueError:
            print(f"Failed to compute feature {x[0].__name__}")
        except ImportError:
            print(f"Failed to import feature {x[0].__name__}")
    return bench_df

## 3. Run benchmarks
---

In [ ]:
# Code to prettify benchmark results
def table_prettifier(df: pl.DataFrame, n: int):
    table = (
        df.filter(pl.col("n") == n)
        .drop("n")
        .sort("speedup", descending=True)
        .with_columns(
            pl.when(pl.exclude("Feature name").abs() < 0.1)
            .then(pl.exclude("Feature name").round(4))
            .when(pl.exclude("Feature name").abs() < 1)
            .then(pl.exclude("Feature name").round(2))
            .when(pl.exclude("Feature name").abs() < 30)
            .then(pl.exclude("Feature name").round(1))
            .otherwise(pl.exclude("Feature name").round(1))
        )
        .with_columns(speedup="x " + pl.col("speedup").cast(pl.Utf8))
    )
    return table

In [36]:
%%capture
bench_expr = all_benchmarks(params = FUNC_PARAMS_BENCH ,expr = True)
bench_series = all_benchmarks(params = FUNC_PARAMS_BENCH, expr = False)

# Lazy benchmarks
df_expr_10k = table_prettifier(bench_expr, n=10_000)
df_expr_100k = table_prettifier(bench_expr, n=100_000)
df_expr_1m = table_prettifier(bench_expr, n=1_000_000)
df_expr_9m = table_prettifier(bench_expr, n=9_000_000)

# Eager benchmarks
df_series_10k = table_prettifier(bench_series, n=10_000)
df_series_100k = table_prettifier(bench_series, n=100_000)
df_series_1m = table_prettifier(bench_series, n=1_000_000)
df_series_9m = table_prettifier(bench_series, n=9_000_000)

INFO:functime.feature_extraction.tsfresh:Expression version of approximate_entropy is not yet implemented due to technical difficulty regarding Polars Expression Plugins.
INFO:functime.feature_extraction.tsfresh:Expression version of autoregressive_coefficients is not yet implemented due to technical difficulty regarding Polars Expression Plugins.
INFO:functime.feature_extraction.tsfresh:Expression version of sample_entropy is not yet implemented due to technical difficulty regarding Polars Expression Plugins.
INFO:functime.feature_extraction.tsfresh:Expression version of spkt_welch_density is not yet implemented due to technical difficulty regarding Polars Expression Plugins.


## 4. Benchmark results
---

Display 8 tables:
- For `pl.Series`: 10k, 100k, 1M and 9M rows
- For `pl.Expr`: 10k, 100k, 1M and 9M rows

Each table contains the execution time (ms) for tsfresh and functime, the difference, the difference in % and the speedup:

### 4.1 Results for `pl.Expr`

#### 10k expr

In [41]:
df_expr_10k

Feature name,functime (ms),tfresh (ms),diff (ms),diff %,speedup
"""benford_correlation2""",0.32,5.8,-5.4,-94.4,"""x 17.9"""
"""benford_correlation""",0.56,5.7,-5.1,-90.1,"""x 10.1"""
"""mean_n_absolute_max""",0.0393,0.39,-0.35,-89.9,"""x 9.9"""
"""energy_ratios""",0.16,1.0,-0.88,-84.4,"""x 6.4"""
"""longest_streak_below_mean""",0.15,0.61,-0.46,-75.0,"""x 4.0"""
"""large_standard_deviation""",0.0192,0.0703,-0.0512,-72.8,"""x 3.7"""
"""range_count""",0.018,0.0657,-0.0478,-72.7,"""x 3.7"""
"""longest_streak_above_mean""",0.17,0.59,-0.42,-70.7,"""x 3.4"""
"""change_quantiles""",0.17,0.5,-0.32,-64.9,"""x 2.8"""
"""var_gt_std""",0.0132,0.0368,-0.0236,-64.2,"""x 2.8"""


#### 100k expr

In [42]:
df_expr_100k

Feature name,functime (ms),tfresh (ms),diff (ms),diff %,speedup
"""mean_n_absolute_max""",0.19,5.2,-5.0,-96.4,"""x 27.6"""
"""benford_correlation2""",2.2,58.4,-56.3,-96.3,"""x 27.1"""
"""benford_correlation""",5.0,57.6,-52.6,-91.3,"""x 11.5"""
"""var_gt_std""",0.0438,0.29,-0.25,-85.1,"""x 6.7"""
"""variation_coefficient""",0.0571,0.35,-0.29,-83.6,"""x 6.1"""
"""longest_streak_below_mean""",1.2,5.9,-4.7,-80.0,"""x 5.0"""
"""longest_streak_above_mean""",1.2,5.9,-4.7,-79.9,"""x 5.0"""
"""change_quantiles""",0.75,3.5,-2.7,-78.3,"""x 4.6"""
"""large_standard_deviation""",0.15,0.53,-0.39,-72.7,"""x 3.7"""
"""energy_ratios""",0.37,1.3,-0.9,-70.8,"""x 3.4"""


#### 1M expr

In [43]:
df_expr_1m

Feature name,functime (ms),tfresh (ms),diff (ms),diff %,speedup
"""benford_correlation2""",21.3,588.6,-567.3,-96.4,"""x 27.6"""
"""mean_n_absolute_max""",2.5,61.5,-59.0,-96.0,"""x 24.7"""
"""absolute_maximum""",0.0792,1.6,-1.5,-95.0,"""x 19.9"""
"""benford_correlation""",51.3,592.9,-541.6,-91.3,"""x 11.6"""
"""large_standard_deviation""",0.5,5.2,-4.7,-90.4,"""x 10.4"""
"""variation_coefficient""",0.35,3.3,-3.0,-89.3,"""x 9.4"""
"""var_gt_std""",0.36,2.9,-2.5,-87.4,"""x 7.9"""
"""has_duplicate_min""",0.24,1.4,-1.2,-83.7,"""x 6.1"""
"""has_duplicate_max""",0.25,1.4,-1.2,-82.6,"""x 5.8"""
"""longest_streak_above_mean""",11.2,59.6,-48.3,-81.1,"""x 5.3"""


#### 9M expr

In [44]:
df_expr_9m

Feature name,functime (ms),tfresh (ms),diff (ms),diff %,speedup
"""mean_n_absolute_max""",16.1,601.7,-585.6,-97.3,"""x 37.4"""
"""benford_correlation2""",201.0,5350.0,-5149.0,-96.2,"""x 26.6"""
"""absolute_maximum""",0.97,16.6,-15.6,-94.1,"""x 17.1"""
"""benford_correlation""",458.0,5365.4,-4907.4,-91.5,"""x 11.7"""
"""large_standard_deviation""",5.4,47.2,-41.8,-88.5,"""x 8.7"""
"""variation_coefficient""",4.2,31.6,-27.3,-86.6,"""x 7.5"""
"""number_peaks""",15.0,102.0,-87.0,-85.3,"""x 6.8"""
"""has_duplicate_min""",2.1,14.0,-11.9,-85.2,"""x 6.8"""
"""has_duplicate_max""",2.1,13.9,-11.9,-85.1,"""x 6.7"""
"""var_gt_std""",4.0,26.0,-22.1,-84.8,"""x 6.6"""


### 4.2 Results for `pl.Series`

#### 10k series

In [45]:
df_series_10k

Feature name,functime (ms),tfresh (ms),diff (ms),diff %,speedup
"""approximate_entropy""",43.1,5688.4,-5645.2,-99.2,"""x 131.9"""
"""sample_entropy""",35.1,3294.3,-3259.2,-98.9,"""x 93.8"""
"""benford_correlation2""",0.1,5.8,-5.7,-98.2,"""x 56.3"""
"""energy_ratios""",0.0865,1.0,-0.96,-91.7,"""x 12.1"""
"""benford_correlation""",0.62,5.9,-5.3,-89.5,"""x 9.5"""
"""mean_n_absolute_max""",0.0515,0.38,-0.33,-86.5,"""x 7.4"""
"""has_duplicate_min""",0.0067,0.0473,-0.0405,-85.8,"""x 7.0"""
"""has_duplicate_max""",0.0067,0.047,-0.0403,-85.7,"""x 7.0"""
"""count_below_mean""",0.0065,0.0425,-0.036,-84.6,"""x 6.5"""
"""count_above_mean""",0.0065,0.0421,-0.0355,-84.5,"""x 6.4"""


#### 100k series

In [46]:
df_series_100k

Feature name,functime (ms),tfresh (ms),diff (ms),diff %,speedup
"""sample_entropy""",511.1,347925.6,-347414.5,-99.9,"""x 680.8"""
"""benford_correlation2""",0.4,58.0,-57.6,-99.3,"""x 146.4"""
"""mean_n_absolute_max""",0.19,5.2,-5.0,-96.3,"""x 27.4"""
"""benford_correlation""",5.0,58.2,-53.2,-91.4,"""x 11.7"""
"""large_standard_deviation""",0.0621,0.53,-0.47,-88.3,"""x 8.6"""
"""has_duplicate_max""",0.0232,0.17,-0.15,-86.6,"""x 7.5"""
"""has_duplicate_min""",0.0233,0.17,-0.15,-86.6,"""x 7.5"""
"""linear_trend""",0.46,3.4,-3.0,-86.5,"""x 7.4"""
"""energy_ratios""",0.19,1.3,-1.1,-85.5,"""x 6.9"""
"""absolute_maximum""",0.0213,0.15,-0.12,-85.4,"""x 6.9"""


#### 1M series

In [47]:
df_series_1m

Feature name,functime (ms),tfresh (ms),diff (ms),diff %,speedup
"""benford_correlation2""",3.4,588.9,-585.5,-99.4,"""x 172.4"""
"""mean_n_absolute_max""",2.6,61.2,-58.6,-95.8,"""x 23.7"""
"""benford_correlation""",51.0,589.7,-538.7,-91.4,"""x 11.6"""
"""large_standard_deviation""",0.55,5.2,-4.6,-89.3,"""x 9.3"""
"""var_gt_std""",0.35,2.9,-2.5,-87.9,"""x 8.3"""
"""absolute_maximum""",0.18,1.4,-1.2,-87.2,"""x 7.8"""
"""has_duplicate_max""",0.19,1.4,-1.3,-87.1,"""x 7.8"""
"""has_duplicate_min""",0.19,1.4,-1.3,-87.1,"""x 7.8"""
"""linear_trend""",5.0,38.3,-33.3,-87.0,"""x 7.7"""
"""count_below_mean""",0.2,1.4,-1.2,-86.3,"""x 7.3"""


#### 9M series

In [48]:
df_series_9m

Feature name,functime (ms),tfresh (ms),diff (ms),diff %,speedup
"""benford_correlation2""",31.4,5368.5,-5337.1,-99.4,"""x 171.2"""
"""mean_n_absolute_max""",13.6,599.5,-585.9,-97.7,"""x 44.0"""
"""benford_correlation""",511.1,5381.9,-4870.7,-90.5,"""x 10.5"""
"""large_standard_deviation""",6.4,47.1,-40.6,-86.4,"""x 7.3"""
"""linear_trend""",50.5,368.1,-317.6,-86.3,"""x 7.3"""
"""absolute_maximum""",2.3,16.1,-13.8,-85.7,"""x 7.0"""
"""number_peaks""",15.1,102.9,-87.7,-85.3,"""x 6.8"""
"""var_gt_std""",3.9,26.1,-22.2,-85.0,"""x 6.7"""
"""ratio_n_unique_to_length""",94.0,610.2,-516.2,-84.6,"""x 6.5"""
"""energy_ratios""",11.0,70.1,-59.1,-84.3,"""x 6.4"""


## 5. Benchmark `Group by / Aggregation` context

Benchmark combining functime's feature extraction and polars' `Group by / Aggregation` context.

In [49]:
_SP500_DATASET = "../../data/sp500.parquet"

SP500_PANDAS = pd.read_parquet(_SP500_DATASET)
SP500_PL_EAGER = pl.read_parquet(_SP500_DATASET)

In [50]:
SP500_PANDAS

,ticker,time,price
0,A,2022-06-01,122.278214
1,A,2022-06-02,128.248581
2,A,2022-06-03,127.642609
3,A,2022-06-06,126.788277
4,A,2022-06-07,128.049881
...,...,...,...
126248,ZTS,2023-05-24,169.139999
126249,ZTS,2023-05-25,165.240005
126250,ZTS,2023-05-26,164.740005
126251,ZTS,2023-05-30,160.940002


We want to compare `tsfresh` using `pandas' groupby`  with  `functime` using `polars' groupby` such as:

In [51]:
%%timeit
SP500_PANDAS.groupby(
    by = "ticker"
)["price"].agg(
    tsfresh.number_peaks,
    n = 5
)

209 ms ± 673 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%%timeit
SP500_PL_EAGER.group_by(
    pl.col("ticker")
).agg(
    pl.col("price").ts.number_peaks(support = 5)
)

22.1 ms ± 522 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


If we examine the previous benchmark, we can see that the `number_peaks` operation is approximately **2.5** times faster when using `functime` compared to `tsfresh`.

In the `groupby` context, it's **10** times faster!

In [53]:
def benchmark_groupby_context(
    f_feat: Callable, ts_feat: Callable, f_params: dict, ts_params: dict
):
    benchmark = perfplot.bench(
        setup=lambda _n: (SP500_PL_EAGER, SP500_PANDAS),
        kernels=[
            lambda x, _y: x.group_by(pl.col("ticker")).agg(
                f_feat(pl.col("price"), **f_params)
            ),  # functime + polars groupby
            lambda _x, y: y.groupby("ticker")["price"].agg(
                ts_feat, **ts_params
            ),  # tsfresh + pandas groupby
        ],
        n_range=[1],
        equality_check=False,
        labels=["functime", "tsfresh"],
    )
    return benchmark

In [54]:
def all_benchmarks_groupby(params: list[tuple]) -> list:
    bench_df = pl.DataFrame(
        schema={
            "Feature name": pl.Utf8,
            "n": pl.Int64,
            "functime + pl groupby (ms)": pl.Float64,
            "tfresh + pd groupby (ms)": pl.Float64,
            "diff (ms)": pl.Float64,
            "diff %": pl.Float64,
            "speedup": pl.Float64,
        }
    )
    for x in params:
        try:
            print(f"Feature: {x[0].__name__}")
            bench = benchmark_groupby_context(
                f_feat=x[0], ts_feat=x[1], f_params=x[2], ts_params=x[3]
            )
            bench_df = pl.concat(
                [
                    pl.DataFrame(
                        {
                            "Feature name": [x[0].__name__] * len(bench.n_range),
                            "n": bench.n_range,
                            "functime + pl groupby (ms)": bench.timings_s[0] * 1_000,
                            "tfresh + pd groupby (ms)": bench.timings_s[1] * 1_000,
                            "diff (ms)": (bench.timings_s[0] - bench.timings_s[1])
                            * 1_000,
                            "diff %": 100
                            * (bench.timings_s[0] - bench.timings_s[1])
                            / bench.timings_s[1],
                            "speedup": bench.timings_s[1] / bench.timings_s[0],
                        }
                    ),
                    bench_df,
                ]
            )
        except ValueError:
            print(f"Failed to compute feature {x[0].__name__}")
        except ImportError:
            print(f"Failed to import feature {x[0].__name__}")
    return bench_df

In [58]:
%%capture
bench_groupby = all_benchmarks_groupby(params=FUNC_PARAMS_BENCH)
df_groupby = table_prettifier(df=bench_groupby, n=1)

INFO:functime.feature_extraction.tsfresh:Expression version of approximate_entropy is not yet implemented due to technical difficulty regarding Polars Expression Plugins.
INFO:functime.feature_extraction.tsfresh:Expression version of autoregressive_coefficients is not yet implemented due to technical difficulty regarding Polars Expression Plugins.
The predicate '[(col("price").abs()) == (1000.0)]' in 'when->then->otherwise' is not a valid aggregation and might produce a different number of rows than the group_by operation would. This behavior is experimental and may be subject to change
INFO:functime.feature_extraction.tsfresh:Expression version of sample_entropy is not yet implemented due to technical difficulty regarding Polars Expression Plugins.
INFO:functime.feature_extraction.tsfresh:Expression version of spkt_welch_density is not yet implemented due to technical difficulty regarding Polars Expression Plugins.


#### S&P500 groupby

In [57]:
df_groupby

Feature name,functime + pl groupby (ms),tfresh + pd groupby (ms),diff (ms),diff %,speedup
"""energy_ratios""",2.7,634.9,-632.3,-99.6,"""x 238.8"""
"""range_count""",1.1,38.9,-37.8,-97.2,"""x 35.4"""
"""percent_reoccurring_points""",2.6,64.4,-61.9,-96.0,"""x 25.1"""
"""root_mean_square""",0.91,22.5,-21.6,-96.0,"""x 24.7"""
"""symmetry_looking""",1.1,25.3,-24.3,-95.8,"""x 23.7"""
"""ratio_beyond_r_sigma""",2.2,50.7,-48.5,-95.7,"""x 23.2"""
"""count_above""",0.9,19.2,-18.3,-95.3,"""x 21.3"""
"""change_quantiles""",5.7,114.9,-109.2,-95.0,"""x 20.0"""
"""count_below""",0.95,19.0,-18.1,-95.0,"""x 20.0"""
"""cid_ce""",2.3,40.9,-38.6,-94.4,"""x 17.9"""
